# Blink Project: Automated Data Processing Notebook

This notebook uses a two-run workflow to transform raw survey data files (`.var`, `.dat`, `.o`) into a clean, analysis-ready dataset.

## Workflow Instructions

### Run 1: Setup Mode
1.  Place your raw `.var`, `.dat`, and `.o` files in a folder.
2.  In the final code cell, set `DATA_DIRECTORY` to that folder's path and `ID_COLUMN_NAME` to your survey's unique ID.
3.  Set `RUN_MODE = 'SETUP'`.
4.  Run all cells in the notebook.
5.  The script will generate **one single Excel file** in your data directory: **`config_and_templates.xlsx`**.

### Configuration Step (User's Task)
Open the `config_and_templates.xlsx` file and edit the sheets as needed:

1.  **`QuestionList` Sheet**:
    * In the **`Keep`** column, type `yes` for variables you want in the final dataset.
    * In the **`ChartName`** column, provide a user-friendly name for reporting.
2.  **`LabelMap` Sheet**: Use the `CodeList` sheet as a reference to add any label revisions (`VariableName`, `OriginalLabel`, `RevisedLabel`).
3.  **`RespondentFilter` Sheet**: (Optional) Paste a list of respondent IDs (matching the `ID_COLUMN_NAME`) that you want to **keep**. If this sheet is left blank, all respondents will be included.
4.  **`ExtractList` Sheet**: (Optional) List the final `ChartName`s you want to export for a quick review.
5.  **`Corrections` Sheet**: (Optional) Add any row-specific data corrections.

### Run 2: Process Mode
1.  In the final code cell, change `RUN_MODE = 'PROCESS'`.
2.  Run all the notebook cells again.
3.  The script will use your single, configured Excel file to generate the final outputs.

## Imports and Helper Functions

This cell contains utility functions for file handling, configuration, and Excel formatting.

In [ ]:
import pandas as pd
from dataclasses import dataclass, field
from typing import Dict, List
import os
import warnings

# Suppress the specific FutureWarning from pandas to keep the output clean
warnings.simplefilter(action='ignore', category=FutureWarning)

# This class will hold the information for each variable
@dataclass
class VarDefinition:
    name: str
    type: str
    position: int
    length: int
    question_text: str
    labels: Dict[str, str] = field(default_factory=dict)

def get_file_paths(directory: str) -> dict:
    """Scans a directory and finds the first .var, .dat, and .o files."""
    paths = {'var': None, 'dat': None, 'o': None}
    print(f"Scanning directory: {directory}")
    try:
        for filename in os.listdir(directory):
            filepath = os.path.join(directory, filename)
            if not paths['var'] and filename.lower().endswith('.var'): paths['var'] = filepath
            elif not paths['dat'] and filename.lower().endswith('.dat'): paths['dat'] = filepath
            elif not paths['o'] and filename.lower().endswith('.o'): paths['o'] = filepath
    except FileNotFoundError:
        print(f"❌ Error: Directory not found at '{directory}'"); return None
    return paths

def _format_excel_sheet(writer: pd.ExcelWriter, df: pd.DataFrame, sheet_name: str):
    """Internal helper to format a single sheet within an ExcelWriter object."""
    df.to_excel(writer, sheet_name=sheet_name, index=False)
    workbook = writer.book
    worksheet = writer.sheets[sheet_name]
    header_format = workbook.add_format({'bold': True, 'bg_color': '#D3D3D3'})
    
    for col_num, value in enumerate(df.columns.values):
        worksheet.write(0, col_num, value, header_format)
    
    for i, col in enumerate(df.columns):
        column_len = len(col)
        if not df[col].empty:
            max_len = df[col].astype(str).str.len().max()
            column_len = max(column_len, max_len)
        worksheet.set_column(i, i, min(column_len + 2, 60))

def save_df_to_formatted_excel(df: pd.DataFrame, path: str):
    """Saves a single DataFrame to a nicely formatted Excel file."""
    try:
        with pd.ExcelWriter(path, engine='xlsxwriter') as writer:
            _format_excel_sheet(writer, df, 'Sheet1')
    except Exception as e:
        print(f"  -> Warning: Could not write formatted Excel file '{os.path.basename(path)}'. Error: {e}")

def load_config_sheet(config_path: str, sheet_name: str) -> pd.DataFrame:
    """Loads a specific sheet from the config file, returning an empty DataFrame if not found."""
    try:
        df = pd.read_excel(config_path, sheet_name=sheet_name)
        return df.dropna(how='all')
    except FileNotFoundError:
        return None
    except ValueError:
        print(f"ⓘ Sheet '{sheet_name}' not found in config file. Skipping this step.")
        return pd.DataFrame()
    except Exception as e:
        print(f"  -> Warning: Could not read sheet '{sheet_name}'. Error: {e}")
        return pd.DataFrame()

## Parse Variables and Create Config Files

This is the most critical step. The `.var` file acts as a "codebook" or "map" that defines the structure of the main data file. This function reads and interprets that map.

**File Format:**

The script understands four types of lines in the UTF-16 encoded `.var` file:

- Variable Definition Lines: Always start with an asterisk (`*`) and define the variable's name, type (e.g., `*SNG`, `*NUM`), position, and length.

- Code Frame/Label Lines: Start with a number (the code), followed by a colon, and then the text label (e.g., `1: Yes`).

- Multiline Question Text: Lines starting with several spaces are treated as a continuation of the previous variable's question text.

- Special `QTYPE:OPEN` Lines: This line overrides the preceding variable's type to `OPN` (Open-End).

Additionaly, the script will generate the `config_and_templates.xlsx` file for the users to input

In [ ]:
def parse_var_file(filepath: str) -> List[VarDefinition]:
    """Reads a .var file and returns a list of VarDefinition objects."""
    print(f"⚙️ Parsing `{os.path.basename(filepath)}`...")
    # (This function is correct and unchanged)
    var_definitions: List[VarDefinition] = []
    current_variable: VarDefinition = None
    with open(filepath, 'r', encoding='utf-16') as f:
        for line in f:
            line_stripped = line.strip()
            if not line_stripped: continue
            if line_stripped.startswith('*'):
                try:
                    definition_part, question_text = line_stripped.split(':', 1)
                    parts = definition_part.strip().split()
                    pos_len_part = parts[-1]
                    position, length = map(int, pos_len_part.split('L'))
                    if len(parts) >= 3: var_name, var_type = parts[0][1:], parts[1][1:]
                    else: var_name, var_type = parts[0][1:], "NUM"
                    current_variable = VarDefinition(name=var_name, type=var_type, position=position, length=length, question_text=question_text.strip())
                    var_definitions.append(current_variable)
                except (ValueError, IndexError):
                    print(f"  Warning: Could not parse variable line: {line_stripped}")
                    current_variable = None
            elif "QTYPE:OPEN" in line_stripped:
                if current_variable: current_variable.type = 'OPN'
            elif ':' in line_stripped and line_stripped.split(':', 1)[0].strip().isdigit():
                if current_variable:
                    try:
                        code, label = line_stripped.split(':', 1)
                        current_variable.labels[code.strip()] = label.strip()
                    except ValueError:
                        print(f"  Warning: Could not parse label line: {line_stripped}")
            elif line.startswith('   ') and current_variable:
                current_variable.question_text += " " + line_stripped
    return var_definitions

def export_setup_files(parsed_vars: List[VarDefinition], output_path: str, id_column_name: str, filenames: dict, dat_filepath: str):
    """Exports a single, multi-sheet Excel file with all templates, ensuring RespondentList has numeric IDs."""
    print(f"⚙️ Exporting setup files to a single workbook: `{os.path.basename(output_path)}`...")
    try:
        all_respondent_ids = []
        if dat_filepath:
            print("  -> Reading respondent IDs from .dat file for the list...")
            id_var = next((var for var in parsed_vars if var.name == id_column_name), None)
            if id_var:
                colspec = [(id_var.position - 1, id_var.position - 1 + id_var.length)]
                df_ids = pd.read_fwf(dat_filepath, colspecs=colspec, header=None, encoding='utf-16', dtype=str)
                # Convert IDs to a clean, numeric list for the template
                all_respondent_ids = pd.to_numeric(df_ids.iloc[:, 0], errors='coerce').dropna().astype(int).unique().tolist()
        
        with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
            sheets_to_create = {
                filenames['qlist']: pd.DataFrame([{'VariableName': v.name, 'Type': v.type, 'QuestionText': v.question_text, 'Keep': '', 'ChartName': ''} for v in parsed_vars]),
                filenames['codelist']: pd.DataFrame([{'VariableName': v.name, 'Type': v.type, 'QuestionText': v.question_text, 'LabelCode': c, 'OriginalLabel': l} for v in parsed_vars if v.labels for c, l in v.labels.items()]),
                filenames['label_map']: pd.DataFrame(columns=['VariableName', 'OriginalLabel', 'RevisedLabel']),
                filenames['respondent_list']: pd.DataFrame({id_column_name: all_respondent_ids}),
                filenames['respondent_filter']: pd.DataFrame(columns=[id_column_name]),
                filenames['extract_list']: pd.DataFrame(columns=['ChartName']),
                filenames['corrections']: pd.DataFrame(columns=[id_column_name, 'VariableName', 'NewValue'])
            }
            for sheet_name, df in sheets_to_create.items():
                _format_excel_sheet(writer, df, sheet_name)
    except Exception as e:
        print(f"  -> ERROR: Could not write the setup file. Error: {e}")

## Build the Final Label Map

This function prepares all the labels upfront. It takes the original labels from the `.var` file and applies your revisions from the LabelMap sheet.

In [ ]:
def build_final_label_map(var_definitions: List[VarDefinition], revision_map: pd.DataFrame) -> dict:
    """Creates a final, revised master map of all labels *before* any data transformation."""
    print("⚙️ Building final label map with user revisions...")
    
    # Start with the original labels from the .var file
    master_label_map = {var.name: var.labels.copy() for var in var_definitions if var.labels}
    
    # Apply revisions from the user's configuration file
    if not revision_map.empty:
        for _, row in revision_map.iterrows():
            var_name = row['VariableName']
            original_label = row['OriginalLabel']
            revised_label = row['RevisedLabel']
            
            # Find the code corresponding to the original label to update the map
            if var_name in master_label_map:
                for code, label in master_label_map[var_name].items():
                    if label == original_label:
                        master_label_map[var_name][code] = revised_label
                        break # Move to the next revision
        print(f"  -> Successfully applied {len(revision_map)} label revisions to the master map.")
    else:
        print("  -> `LabelMap` sheet is empty or not found. Using original labels only.")
        
    return master_label_map

## Read and Process the Coded Data File (`.dat`)

These functions use the map created in Step 1 to read and transform the raw respondent data from the .dat file.

**2.1 read_dat_file**
This function reads the raw data. It constructs lists of column positions (`colspecs`) and column `names` from the parsed variable definitions. It then uses `pandas.read_fwf` to efficiently parse the fixed-width text file, ensuring `encoding='utf-16'` is used. All data is read as text (`dtype=str`) initially to prevent data type errors.

**2.2 filter_by_respondent_id**
This function filters the main DataFrame to only include respondent IDs from a given list

**2.3 process_multi_variables**
This function handles the special logic for Multi-Value (`MV`) variables. An MV variable is a string like `'10100'` where each character's position represents an answer option. The function expands this single column into multiple new columns (e.g., `Q2_1`, `Q2_2`, etc.). For each new column, it checks the corresponding character in the original string: if it's `'1'`, the cell gets the text label; otherwise, it's left blank.

**2.4 translate_codes_to_labels**
This funtion translates all codes to their original labels from the .var file, then applies if revisions from the label map file

In [ ]:
def read_dat_file(dat_filepath: str, var_definitions: List[VarDefinition], id_column: str) -> pd.DataFrame:
    """Reads a fixed-width .dat file and immediately converts the ID column to a numeric type."""
    colspecs, names = [], []
    for var in var_definitions:
        if var.type:
            start_pos, end_pos = var.position - 1, var.position - 1 + var.length
            colspecs.append((start_pos, end_pos))
            names.append(var.name)
    print(f"⚙️ Reading `{os.path.basename(dat_filepath)}`...")
    df = pd.read_fwf(dat_filepath, colspecs=colspecs, header=None, names=names, encoding='utf-16', dtype=str)
    
    print(f"  -> Converting ID column '{id_column}' to a numeric type.")
    df[id_column] = pd.to_numeric(df[id_column], errors='coerce')
    return df

def filter_by_respondent_id(dataframe: pd.DataFrame, respondent_list_df: pd.DataFrame, id_column: str) -> pd.DataFrame:
    """Filters the main DataFrame to only include respondent IDs from a given list."""
    print("⚙️ Filtering by respondent ID list...")
    if respondent_list_df.empty:
        print("  -> Respondent filter list is empty. Keeping all respondents."); return dataframe
    
    # Convert IDs from the Excel file to a numeric type for a reliable match
    ids_to_keep = pd.to_numeric(respondent_list_df.iloc[:, 0], errors='coerce').dropna()
    
    original_rows = len(dataframe)
    df_filtered = dataframe[dataframe[id_column].isin(ids_to_keep)].copy()
    print(f"  -> Kept {len(df_filtered)} of {original_rows} respondents based on the filter list.")
    return df_filtered

def process_multi_variables(dataframe: pd.DataFrame, var_definitions: List[VarDefinition], master_label_map: dict) -> pd.DataFrame:
    """Expands MV variables using the final, revised master label map."""
    print("⚙️ Processing Multi-Value (MV) variables...")
    # (The logic for expanding columns is the same, but it now uses the master_label_map for its values)
    original_cols = dataframe.columns.tolist()
    new_column_dfs = []
    mv_vars = [var for var in var_definitions if var.type == 'MV']
    
    for var in mv_vars:
        if var.name not in original_cols: continue
        
        final_labels = master_label_map.get(var.name, {})
        if not final_labels: continue
        
        new_cols_dict = {}
        for code, label in final_labels.items():
            new_col_name = f"{var.name}_{code}"
            try:
                char_index = int(code) - 1
                if char_index < 0: continue
                # THE FIX: It now uses the potentially revised 'label' from the master map
                new_cols_dict[new_col_name] = dataframe[var.name].str[char_index].apply(lambda char: label if char == '1' else '')
            except (IndexError, ValueError):
                print(f"  Warning: Could not process code '{code}' for variable '{var.name}'.")
                continue
        if new_cols_dict:
            new_column_dfs.append(pd.DataFrame(new_cols_dict))
        original_cols.remove(var.name)
        
    return pd.concat([dataframe[original_cols]] + new_column_dfs, axis=1)

def translate_codes_to_labels(dataframe: pd.DataFrame, var_definitions: List[VarDefinition], master_label_map: dict) -> pd.DataFrame:
    """Translates codes to labels for SNG variables using the final master label map."""
    print("⚙️ Translating codes to labels for Single-Value (SNG) variables...")
    
    # THE FIX: This function is now much simpler. It only handles SNG variables.
    sng_vars = {var.name for var in var_definitions if var.type == 'SNG'}
    
    for col_name in sng_vars:
        if col_name in dataframe.columns and col_name in master_label_map:
            # Replace codes with the final, revised labels from the master map
            dataframe[col_name] = dataframe[col_name].str.strip().replace(master_label_map[col_name])
            
    return dataframe

## Process and Merge the Open-Ended Data (`.o`) File

This is the final data integration step, where verbatim text answers are added to the dataset.

**4a. parse_o_file**
The `.o` file contains both standard and "extended" formats for its lines. This function reads the file line by line (not with `read_fwf`) to handle this complexity. It checks each line for the extended format marker (`*`) to determine the correct character positions for the respondent ID, the question's original position, and the answer text. It returns a clean DataFrame of the parsed open-ended data.

**4b. merge_o_data**
This function merges the parsed open-ended data into the main DataFrame. It first creates a quick lookup map to get a variable's name from its position (`pos_to_name_map`). It then iterates through each open-ended answer, using the respondent ID and the mapped variable name to find the exact cell (`.loc`) in the main DataFrame to place the text.

In [ ]:
def parse_o_file(o_filepath: str) -> pd.DataFrame:
    """Reads the .o file line-by-line to handle standard and extended formats."""
    print(f"⚙️ Parsing `{os.path.basename(o_filepath)}` for open-ended answers...")
    parsed_records = []
    try:
        with open(o_filepath, 'r', encoding='utf-16') as f:
            for line in f:
                if len(line) < 22: continue
                respondent_id, answer_text, question_pos_str = line[0:8].strip(), "", ""
                if line[10:11] == '*':
                    question_pos_str, answer_text = line[11:21].strip(), line[31:].strip()
                else:
                    question_pos_str, answer_text = line[10:15].strip(), line[18:].strip()
                if question_pos_str.isdigit():
                    parsed_records.append({'id': respondent_id, 'position': int(question_pos_str), 'text': answer_text})
    except FileNotFoundError:
        print("  -> .o file not found, skipping."); return pd.DataFrame()
    return pd.DataFrame(parsed_records)

def merge_o_data(main_df: pd.DataFrame, df_o: pd.DataFrame, var_definitions: List[VarDefinition], id_column: str) -> pd.DataFrame:
    """
    Merges the parsed open-ended data into the main DataFrame using a robust
    and simple update method.
    """
    if df_o.empty: return main_df
    print("⚙️ Merging open-ended data into main table...")
    
    # 1. Create a map to convert question position to variable name
    pos_to_name_map = {var.position: var.name for var in var_definitions}
    
    # 2. Prepare the open-ended data
    df_o['id'] = pd.to_numeric(df_o['id'], errors='coerce')
    df_o['VariableName'] = df_o['position'].map(pos_to_name_map)
    df_o.dropna(subset=['id', 'VariableName'], inplace=True)
    
    # 3. Pivot the data so there is one row per respondent, with columns for each OPN variable
    df_o_pivoted = df_o.pivot(index='id', columns='VariableName', values='text')
    
    # 4. THE FIX: Use a robust 'update' method
    # This aligns the data on the respondent ID and fills in the blanks in the main
    # DataFrame with the text from the open-ended data, without dropping any columns.
    main_df_indexed = main_df.set_index(id_column)
    main_df_indexed.update(df_o_pivoted)
    main_df = main_df_indexed.reset_index()
    
    print(f"  -> Successfully merged open-ended answers.")
    return main_df

## Apply Manual Corrections (optional)

In [ ]:
def apply_manual_corrections(dataframe: pd.DataFrame, corrections_df: pd.DataFrame, id_column: str) -> pd.DataFrame:
    """Applies manual data corrections using numeric IDs."""
    print("⚙️ Applying manual data corrections...")
    if corrections_df.empty:
        print("  -> Correction sheet is empty. Skipping."); return dataframe
    
    df = dataframe.copy()
    # THE FIX: Ensure IDs from the corrections file are also numeric for matching
    corrections_df[id_column] = pd.to_numeric(corrections_df[id_column], errors='coerce')

    # Set the ID as the index for efficient updating
    df.set_index(id_column, inplace=True)
    
    for _, row in corrections_df.iterrows():
        respondent_id = row[id_column]
        var_name = row['VariableName']
        new_value = row['NewValue']
        
        # Check if the respondent and variable exist before trying to update
        if pd.notna(respondent_id) and respondent_id in df.index and var_name in df.columns:
            df.loc[respondent_id, var_name] = new_value
            
    print(f"  -> Applied {len(corrections_df)} corrections.")
    return df.reset_index()

## Custom Transformations (optional)
This cell is for creating new, calculated variables (derived variables) from the cleaned data. Add your project-specific transformations here *before* running the final cell. This function will be called during the `'PROCESS'` run.

In [ ]:
def create_derived_variables(df: pd.DataFrame) -> pd.DataFrame:
    """Creates new columns in the DataFrame based on custom calculations."""
    print("⚙️ Creating derived variables...")

    # --- Add your custom logic below ---

    # Example 1: Create Age Groups from an 'AGE' column
    if 'AGE' in df.columns:
        age_numeric = pd.to_numeric(df['AGE'], errors='coerce')
        bins = [0, 24, 39, 54, 100]
        labels = ['18-24', '25-39', '40-54', '55+']
        df['Age_Group'] = pd.cut(age_numeric, bins=bins, labels=labels, right=False)
        print("  -> Created 'Age_Group' column.")

    # Example 2: Combine 'Awareness' columns into a total score
    awareness_cols = [col for col in df.columns if 'Aware_' in col and col.endswith('_1')]
    if awareness_cols:
        for col in awareness_cols:
             df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
        df['Total_Awareness'] = df[awareness_cols].sum(axis=1)
        print("  -> Created 'Total_Awareness' column.")
        
    # --- End of custom logic ---
    
    return df

## Extract Selected Data to Excel (optional)

The final step is to save the results. The main script saves the complete, cleaned dataset. This helper function provides an optional way to save a smaller subset of columns to a separate Excel file for quick review.

In [ ]:
def extract_data_to_excel(dataframe: pd.DataFrame, extract_list_df: pd.DataFrame, output_path: str):
    """
    Exports a subset of columns to a separate, formatted Excel file for review.
    UPDATED: Now correctly handles requests for Multi-Value (MV) variables by looking for
    column names that start with the requested ChartName.
    """
    if extract_list_df.empty or extract_list_df.iloc[:, 0].dropna().empty:
        print("ⓘ Extract list is empty, skipping data extraction."); return
        
    chart_names_to_extract = extract_list_df.iloc[:, 0].dropna().tolist()
    print(f"⚙️ Exporting {len(chart_names_to_extract)} specified variables to `{os.path.basename(output_path)}`...")
    
    final_cols_to_extract = []
    found_chart_names = set()

    # Iterate through the user's requested chart names and find all matching columns
    for name in chart_names_to_extract:
        # Find exact matches (for SNG, OPN, etc.)
        if name in dataframe.columns:
            final_cols_to_extract.append(name)
            found_chart_names.add(name)
        
        # Find partial matches for expanded MV columns (e.g., "Aided Awareness_51")
        mv_matches = [col for col in dataframe.columns if col.startswith(f"{name}_")]
        if mv_matches:
            final_cols_to_extract.extend(mv_matches)
            found_chart_names.add(name)

    # Report any requested names that didn't match any columns
    missing_names = [name for name in chart_names_to_extract if name not in found_chart_names]
    if missing_names:
        print(f"  -> Warning: The following ChartNames from the extract list were not found: {missing_names}")
    
    if not final_cols_to_extract:
        print("  -> Warning: None of the variables in the extract list could be found."); return
        
    save_df_to_formatted_excel(dataframe[final_cols_to_extract], output_path)
    print(f"  -> Successfully created extract file.")

## Generate Unpivoted MV Files

This function identify which MV variables to unpivot based on user's selections in the QuestionList sheet.

In [ ]:
def generate_unpivoted_mv_files(dataframe: pd.DataFrame, vars_to_keep_df: pd.DataFrame, id_column: str, rename_map: dict, output_dir: str):
    """
    Automatically finds selected MV variables and creates separate, 
    unpivoted (long-format) CSV files for them.
    """
    # THE FIX: Identify which MV variables the user wants to keep
    mv_vars_to_unpivot = vars_to_keep_df[vars_to_keep_df['Type'] == 'MV']['VariableName'].tolist()

    if not mv_vars_to_unpivot:
        print("ⓘ No MV variables selected to unpivot. Skipping this step.")
        return
        
    print(f"⚙️ Generating {len(mv_vars_to_unpivot)} unpivoted MV files...")

    for var_name in mv_vars_to_unpivot:
        # Find all the expanded columns for this MV variable (e.g., 'BE2_51', 'BE2_53', ...)
        expanded_cols = [col for col in dataframe.columns if col.startswith(f"{var_name}_")]
        
        if not expanded_cols:
            print(f"  -> Warning: No expanded columns found for MV variable '{var_name}'. Skipping.")
            continue

        # Get the final, user-friendly ChartName for this variable
        chart_name = rename_map.get(var_name, var_name)
        
        # Use pandas 'melt' to unpivot the data
        df_melted = dataframe.melt(
            id_vars=[id_column],
            value_vars=expanded_cols,
            var_name='OriginalColumn', # Temporary column
            value_name=chart_name      # The final column with the brand names
        )
        
        # Clean up the resulting table
        df_melted = df_melted[df_melted[chart_name] != ''].dropna()
        df_melted = df_melted[[id_column, chart_name]] # Keep only the two required columns
        
        # Save to its own CSV file
        output_filename = f"unpivoted_{var_name}.csv"
        output_path = os.path.join(output_dir, output_filename)
        df_melted.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"  -> Successfully created `{output_filename}` with {len(df_melted)} rows.")

## ▶️ Run the Pipeline
This is the main execution cell. Configure your settings below and run this cell to perform either the **SETUP** or **PROCESS** run.

In [ ]:
# --- 1. SET YOUR CONFIGURATION HERE ---
RUN_MODE = 'PROCESS' # Input either 'SETUP' or 'PROCESS'
DATA_DIRECTORY = r'C:\Users\NguyenGi\OneDrive - Kantar\General\Automation\Blink Project\Input data\Golden 2024\202404-OF-L GOLDEN 2024' # Input path to data folder
ID_COLUMN_NAME = 'INTNR' # Input name of the variable that houses the respondent unique ID, usually the first variable in QuestionList
SAVE_FULL_OUTPUT = True

# --- 2. DEFINE FILENAMES ---
FILENAMES = {
    'config': 'config_and_templates.xlsx',
    'qlist': 'QuestionList',
    'codelist': 'CodeList',
    'label_map': 'LabelMap',
    'respondent_list': 'RespondentList',
    'respondent_filter': 'RespondentFilter',
    'extract_list': 'ExtractList',
    'corrections': 'Corrections',
    'final_output': 'data_final_output.csv',
    'extract_output': 'data_extract.xlsx'
}

# --- 3. RUN THE PIPELINE BASED ON THE SELECTED MODE ---
file_paths = get_file_paths(DATA_DIRECTORY)
config_path = os.path.join(DATA_DIRECTORY, FILENAMES['config'])

if RUN_MODE.upper() == 'SETUP':
    print("\n--- Running in SETUP mode ---")
    if file_paths and file_paths['var']:
        parsed_vars = parse_var_file(file_paths['var'])
        export_setup_files(parsed_vars, config_path, ID_COLUMN_NAME, FILENAMES, file_paths.get('dat'))
        print(f"\n✅ SETUP COMPLETE. Please edit the generated file: '{FILENAMES['config']}'.")
    else:
        print("\n❌ Critical Error: Could not find a .var file.")

elif RUN_MODE.upper() == 'PROCESS':
    print("\n--- Running in PROCESS mode ---")
    if file_paths and file_paths['var'] and file_paths['dat']:
        try:
            # --- Load All Configuration Sheets ---
            df_qlist = load_config_sheet(config_path, FILENAMES['qlist'])
            df_label_map = load_config_sheet(config_path, FILENAMES['label_map'])
            df_respondent_filter = load_config_sheet(config_path, FILENAMES['respondent_filter'])
            df_corrections = load_config_sheet(config_path, FILENAMES['corrections'])
            df_extract_list = load_config_sheet(config_path, FILENAMES['extract_list'])
            
            if df_qlist is None: raise FileNotFoundError(f"'{FILENAMES['config']}' not found.")

            # --- Run Full Pipeline ---
            parsed_vars = parse_var_file(file_paths['var'])

            # --- Build the final, revised label map FIRST ---
            final_label_map = build_final_label_map(parsed_vars, df_label_map)
            
            df_main = read_dat_file(file_paths['dat'], parsed_vars, ID_COLUMN_NAME)
            df_main = filter_by_respondent_id(df_main, df_respondent_filter, ID_COLUMN_NAME)
            
            # Pass the final map to the processing functions
            df_main = process_multi_variables(df_main, parsed_vars, final_label_map)
            df_main = translate_codes_to_labels(df_main, parsed_vars, final_label_map)
            
            if file_paths['o']:
                df_o_parsed = parse_o_file(file_paths['o'])
                if not df_o_parsed.empty:
                    df_main = merge_o_data(df_main, df_o_parsed, parsed_vars, ID_COLUMN_NAME)
            
            df_main = apply_manual_corrections(df_main, df_corrections, ID_COLUMN_NAME)
            df_main = create_derived_variables(df_main)
            print("\n✅ All data processing steps are complete.")

            # --- Filter and Rename Columns ---
            vars_to_keep_df = df_qlist[df_qlist['Keep'].astype(str).str.lower() == 'yes']
            user_selection = vars_to_keep_df['VariableName'].tolist()
            rename_map = vars_to_keep_df.set_index('VariableName')['ChartName'].dropna().to_dict()
            
            final_cols_to_keep = []
            if user_selection:
                print(f"⚙️ Selecting variables based on `{FILENAMES['qlist']}` sheet...")
                mv_bases = {var.name for var in parsed_vars if var.type == 'MV'}
                for var_name in user_selection:
                    if var_name in mv_bases:
                        final_cols_to_keep.extend([col for col in df_main.columns if col.startswith(f"{var_name}_")])
                    elif var_name in df_main.columns:
                        final_cols_to_keep.append(var_name)
                df_final = df_main[final_cols_to_keep].copy()
            else:
                print("ⓘ 'Keep' column is empty in config file. Keeping all variables.")
                df_final = df_main.copy()
            
            # Create a new, expanded rename map for all columns, including MV
            expanded_rename_map = {}
            for original_name, new_name in rename_map.items():
                # Find all columns that start with the original MV name (e.g., "BE2_")
                mv_cols_to_rename = [col for col in df_final.columns if col.startswith(f"{original_name}_")]
                if mv_cols_to_rename:
                    # Create a new name for each expanded column (e.g., "Aided Awareness_51")
                    for col in mv_cols_to_rename:
                        suffix = col.split('_', 1)[1] # Gets the part after the first underscore
                        expanded_rename_map[col] = f"{new_name}_{suffix}"
                elif original_name in df_final.columns:
                    # It's a regular (SNG, OPN, etc.) variable
                    expanded_rename_map[original_name] = new_name
            
            df_final.rename(columns=expanded_rename_map, inplace=True)
            print("⚙️ Final variables selected and renamed.")
            
            # --- Save Final Output(s) ---
            if SAVE_FULL_OUTPUT:
                final_output_path = os.path.join(DATA_DIRECTORY, FILENAMES['final_output'])
                df_final.to_csv(final_output_path, index=False, encoding='utf-8-sig')
                print(f"✅ Full output saved to `{final_output_path}`")
            
            extract_data_to_excel(df_final, df_extract_list, os.path.join(DATA_DIRECTORY, FILENAMES['extract_output']))
            
            # --- Generate Unpivoted MV Files (no change needed here) ---
            generate_unpivoted_mv_files(df_main, vars_to_keep_df, ID_COLUMN_NAME, rename_map, DATA_DIRECTORY)
            
            print("\n✅✅✅ Pipeline Complete! ✅✅✅")
            
            print("\nShowing first 5 rows of the final, processed data:")
            display(df_final.head())
        
        except Exception as e:
            print(f"An unexpected error occurred during processing: {e}")
    else:
        print("\n❌ Critical Error: Could not find the required .var and .dat files.")